# Problem Statement

### Credit Card Lead Prediction

Happy Customer Bank is a mid-sized private bank that deals in all kinds of banking products, like Savings accounts, Current accounts, investment products, credit products, among other offerings.


The bank also cross-sells products to its existing customers and to do so they use different kinds of communication like tele-calling, e-mails, recommendations on net banking, mobile banking, etc. 


In this case, the Happy Customer Bank wants to cross sell its credit cards to its existing customers. The bank has identified a set of customers that are eligible for taking these credit cards.


Now, the bank is looking for your help in identifying customers that could show higher intent towards a recommended credit card, given:

    Customer details (gender, age, region etc.)
    Details of his/her relationship with the bank (Channel_Code,Vintage, 'Avg_Asset_Value etc.)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier


from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.linear_model import LogisticRegression

pd.options.display.max_columns = 200

In [2]:
data = pd.read_csv("train_s3TEQDk.csv")
test = pd.read_csv("test_mSzZ8RL.csv")
print(f"Train shape {data.shape}, Test Shape {test.shape}")

Train shape (245725, 11), Test Shape (105312, 10)


In [3]:
train,valid = train_test_split(data,test_size=0.20,random_state=345,stratify=data['Is_Lead'])
train = train.copy()
valid = valid.copy()
print(f"Train shape {train.shape} Validation shape {valid.shape}")

Train shape (196580, 11) Validation shape (49145, 11)


In [4]:
train.head(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
53078,N3ZQ84QR,Female,46,RG280,Self_Employed,X2,51,No,863584,Yes,0
213644,JWGAMK7P,Male,67,RG258,Other,X2,43,Yes,706126,No,0
131870,CX9NGNQT,Male,46,RG279,Self_Employed,X2,26,Yes,422207,Yes,0


In [5]:
test.head(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,VBENBARO,Male,29,RG254,Other,X1,25,Yes,742366,No
1,CCMEWNKY,Male,43,RG268,Other,X2,49,NaN,925537,No
2,VK3KGA9M,Male,31,RG270,Salaried,X1,14,No,215949,No


In [6]:
valid.head(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
148453,OK9KJGZ2,Female,55,RG268,Self_Employed,X1,37,No,929257,No,0
117997,TTC7CPSI,Male,57,RG283,Self_Employed,X3,87,NaN,909740,No,0
5432,MPUWVRAX,Male,39,RG275,Salaried,X1,8,Yes,961742,Yes,0


In [7]:
train['ID'].nunique()

196580

In [8]:
train.isna().sum()

ID                         0
Gender                     0
Age                        0
Region_Code                0
Occupation                 0
Channel_Code               0
Vintage                    0
Credit_Product         23525
Avg_Account_Balance        0
Is_Active                  0
Is_Lead                    0
dtype: int64

In [9]:
train['Gender'].value_counts(normalize=True)

Male      0.546693
Female    0.453307
Name: Gender, dtype: float64

In [10]:
train['Region_Code'].nunique()

35

In [11]:
train['Occupation'].value_counts(normalize=True)

Self_Employed    0.411532
Salaried         0.292731
Other            0.284866
Entrepreneur     0.010871
Name: Occupation, dtype: float64

In [12]:
train['Channel_Code'].value_counts(normalize=True)

X1    0.421279
X3    0.280359
X2    0.275669
X4    0.022693
Name: Channel_Code, dtype: float64

In [13]:
train['Credit_Product'].value_counts(normalize=True)

No     0.667019
Yes    0.332981
Name: Credit_Product, dtype: float64

In [14]:
train['Avg_Account_Balance'].describe()

count    1.965800e+05
mean     1.129489e+06
std      8.532486e+05
min      2.079000e+04
25%      6.042470e+05
50%      8.954865e+05
75%      1.368733e+06
max      1.035201e+07
Name: Avg_Account_Balance, dtype: float64

In [15]:
train['Is_Active'].value_counts(normalize=True)

No     0.611375
Yes    0.388625
Name: Is_Active, dtype: float64

In [16]:
train['Is_Lead'].value_counts(normalize=True)

0    0.762794
1    0.237206
Name: Is_Lead, dtype: float64

In [17]:
train['Age'].describe()

count    196580.000000
mean         43.864971
std          14.821238
min          23.000000
25%          30.000000
50%          43.000000
75%          54.000000
max          85.000000
Name: Age, dtype: float64

In [18]:
train['Vintage'].describe()

count    196580.000000
mean         46.978121
std          32.346981
min           7.000000
25%          20.000000
50%          32.000000
75%          73.000000
max         135.000000
Name: Vintage, dtype: float64

In [19]:
train.groupby(['Is_Lead'])[['Age','Avg_Account_Balance']].median()

,Age,Avg_Account_Balance
Is_Lead,,
0,38,871158
1,49,980686


In [20]:
train['Credit_Product'] = train['Credit_Product'].fillna('No')
train['Avg_Account_Balance'] = np.log(1+train['Avg_Account_Balance'])

test['Credit_Product'] = test['Credit_Product'].fillna('No')
test['Avg_Account_Balance'] = np.log(1+test['Avg_Account_Balance'])

valid['Credit_Product'] = valid['Credit_Product'].fillna('No')
valid['Avg_Account_Balance'] = np.log(1+valid['Avg_Account_Balance'])


In [21]:
train.sample(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
179539,5BXCNPC8,Male,56,RG284,Self_Employed,X3,61,Yes,13.416296,Yes,1
182366,ACGSDKWX,Female,29,RG283,Salaried,X2,15,Yes,14.188355,No,0
35595,E3HHPGPG,Male,28,RG280,Salaried,X1,19,No,14.058810,No,1


In [22]:
train['Is_Active'] = train['Is_Active'].replace({'No':'N','Yes':'Y'})
test['Is_Active'] = test['Is_Active'].replace({'No':'N','Yes':'Y'})
valid['Is_Active'] = valid['Is_Active'].replace({'No':'N','Yes':'Y'})

In [23]:
cat_cols = ['Gender','Region_Code','Occupation','Channel_Code','Credit_Product','Is_Active']
featured_cols = []
for idx,col in enumerate(cat_cols):
    for sub_col in cat_cols[idx+1:]:
        new_col = f"{col}-{sub_col}"
        featured_cols.append(new_col)
        train[new_col] = train[col] + "-" + train[sub_col]
        test[new_col] = test[col] + "-" + test[sub_col]
        valid[new_col] = valid[col] + "-" + valid[sub_col]

train.sample(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,Gender-Region_Code,Gender-Occupation,Gender-Channel_Code,Gender-Credit_Product,Gender-Is_Active,Region_Code-Occupation,Region_Code-Channel_Code,Region_Code-Credit_Product,Region_Code-Is_Active,Occupation-Channel_Code,Occupation-Credit_Product,Occupation-Is_Active,Channel_Code-Credit_Product,Channel_Code-Is_Active,Credit_Product-Is_Active
133001,FBYFTGSE,Female,27,RG272,Salaried,X1,27,No,13.529990,N,0,Female-RG272,Female-Salaried,Female-X1,Female-No,Female-N,RG272-Salaried,RG272-X1,RG272-No,RG272-N,Salaried-X1,Salaried-No,Salaried-N,X1-No,X1-N,No-N
202987,TEXTYPBW,Female,27,RG254,Self_Employed,X1,26,No,13.546653,N,0,Female-RG254,Female-Self_Employed,Female-X1,Female-No,Female-N,RG254-Self_Employed,RG254-X1,RG254-No,RG254-N,Self_Employed-X1,Self_Employed-No,Self_Employed-N,X1-No,X1-N,No-N
157433,WP5AEJVW,Male,69,RG280,Other,X3,80,No,13.903949,N,0,Male-RG280,Male-Other,Male-X3,Male-No,Male-N,RG280-Other,RG280-X3,RG280-No,RG280-N,Other-X3,Other-No,Other-N,X3-No,X3-N,No-N


In [24]:
all_cat_cols = cat_cols + featured_cols
num_col = ['Age','Vintage','Avg_Account_Balance']
for idx,col in enumerate (all_cat_cols):
    for ind, num in enumerate(num_col):
        print(f"Working Cat Col {col} {idx}/{len(all_cat_cols)}, Num col {num} {ind}/{len(num_col)}")
        grp = train.groupby([col])[num].agg(['mean','median','std'])
        grp = grp.add_prefix(f'{col}-{num}-')
        grp = grp.reset_index()
        train = train.merge(grp,on=[col],how='left')
        test = test.merge(grp,on=[col],how='left')
        valid = valid.merge(grp,on=[col],how='left')

Working Cat Col Gender 0/21, Num col Age 0/3
Working Cat Col Gender 0/21, Num col Vintage 1/3
Working Cat Col Gender 0/21, Num col Avg_Account_Balance 2/3
Working Cat Col Region_Code 1/21, Num col Age 0/3
Working Cat Col Region_Code 1/21, Num col Vintage 1/3
Working Cat Col Region_Code 1/21, Num col Avg_Account_Balance 2/3
Working Cat Col Occupation 2/21, Num col Age 0/3
Working Cat Col Occupation 2/21, Num col Vintage 1/3
Working Cat Col Occupation 2/21, Num col Avg_Account_Balance 2/3
Working Cat Col Channel_Code 3/21, Num col Age 0/3
Working Cat Col Channel_Code 3/21, Num col Vintage 1/3
Working Cat Col Channel_Code 3/21, Num col Avg_Account_Balance 2/3
Working Cat Col Credit_Product 4/21, Num col Age 0/3
Working Cat Col Credit_Product 4/21, Num col Vintage 1/3
Working Cat Col Credit_Product 4/21, Num col Avg_Account_Balance 2/3
Working Cat Col Is_Active 5/21, Num col Age 0/3
Working Cat Col Is_Active 5/21, Num col Vintage 1/3
Working Cat Col Is_Active 5/21, Num col Avg_Account_Bala

In [25]:
all_cat_cols = cat_cols + featured_cols
encoder = LabelEncoder()
for col in all_cat_cols:
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.transform(test[col])
    valid[col] = encoder.transform(valid[col])

In [26]:
train.sample(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,Gender-Region_Code,Gender-Occupation,Gender-Channel_Code,Gender-Credit_Product,Gender-Is_Active,Region_Code-Occupation,Region_Code-Channel_Code,Region_Code-Credit_Product,Region_Code-Is_Active,Occupation-Channel_Code,Occupation-Credit_Product,Occupation-Is_Active,Channel_Code-Credit_Product,Channel_Code-Is_Active,Credit_Product-Is_Active,Gender-Age-mean,Gender-Age-median,Gender-Age-std,Gender-Vintage-mean,Gender-Vintage-median,Gender-Vintage-std,Gender-Avg_Account_Balance-mean,Gender-Avg_Account_Balance-median,Gender-Avg_Account_Balance-std,Region_Code-Age-mean,Region_Code-Age-median,Region_Code-Age-std,Region_Code-Vintage-mean,Region_Code-Vintage-median,Region_Code-Vintage-std,Region_Code-Avg_Account_Balance-mean,Region_Code-Avg_Account_Balance-median,Region_Code-Avg_Account_Balance-std,Occupation-Age-mean,Occupation-Age-median,Occupation-Age-std,Occupation-Vintage-mean,Occupation-Vintage-median,Occupation-Vintage-std,Occupation-Avg_Account_Balance-mean,Occupation-Avg_Account_Balance-median,Occupation-Avg_Account_Balance-std,Channel_Code-Age-mean,Channel_Code-Age-median,Channel_Code-Age-std,Channel_Code-Vintage-mean,Channel_Code-Vintage-median,Channel_Code-Vintage-std,Channel_Code-Avg_Account_Balance-mean,Channel_Code-Avg_Account_Balance-median,Channel_Code-Avg_Account_Balance-std,Credit_Product-Age-mean,Credit_Product-Age-median,Credit_Product-Age-std,Credit_Product-Vintage-mean,Credit_Product-Vintage-median,Credit_Product-Vintage-std,Credit_Product-Avg_Account_Balance-mean,Credit_Product-Avg_Account_Balance-median,Credit_Product-Avg_Account_Balance-std,Is_Active-Age-mean,Is_Active-Age-median,Is_Active-Age-std,Is_Active-Vintage-mean,Is_Active-Vintage-median,Is_Active-Vintage-std,Is_Active-Avg_Account_Balance-mean,Is_Active-Avg_Account_Balance-median,Is_Active-Avg_Account_Balance-std,Gender-Region_Code-Age-mean,Gender-Region_Code-Age-median,Gender-Region_Code-Age-std,Gender-Region_Code-Vintage-mean,Gender-Region_Code-Vintage-median,Gender-Region_Code-Vintage-std,Gender-Region_Code-Avg_Account_Balance-mean,Gender-Region_Code-Avg_Account_Balance-median,Gender-Region_Code-Avg_Account_Balance-std,Gender-Occupation-Age-mean,Gender-Occupation-Age-median,Gender-Occupation-Age-std,Gender-Occupation-Vintage-mean,Gender-Occupation-Vintage-median,Gender-Occupation-Vintage-std,Gender-Occupation-Avg_Account_Balance-mean,Gender-Occupation-Avg_Account_Balance-median,Gender-Occupation-Avg_Account_Balance-std,Gender-Channel_Code-Age-mean,Gender-Channel_Code-Age-median,...,Gender-Credit_Product-Avg_Account_Balance-std,Gender-Is_Active-Age-mean,Gender-Is_Active-Age-median,Gender-Is_Active-Age-std,Gender-Is_Active-Vintage-mean,Gender-Is_Active-Vintage-median,Gender-Is_Active-Vintage-std,Gender-Is_Active-Avg_Account_Balance-mean,Gender-Is_Active-Avg_Account_Balance-median,Gender-Is_Active-Avg_Account_Balance-std,Region_Code-Occupation-Age-mean,Region_Code-Occupation-Age-median,Region_Code-Occupation-Age-std,Region_Code-Occupation-Vintage-mean,Region_Code-Occupation-Vintage-median,Region_Code-Occupation-Vintage-std,Region_Code-Occupation-Avg_Account_Balance-mean,Region_Code-Occupation-Avg_Account_Balance-median,Region_Code-Occupation-Avg_Account_Balance-std,Region_Code-Channel_Code-Age-mean,Region_Code-Channel_Code-Age-median,Region_Code-Channel_Code-Age-std,Region_Code-Channel_Code-Vintage-mean,Region_Code-Channel_Code-Vintage-median,Region_Code-Channel_Code-Vintage-std,Region_Code-Channel_Code-Avg_Account_Balance-mean,Region_Code-Channel_Code-Avg_Account_Balance-median,Region_Code-Channel_Code-Avg_Account_Balance-std,Region_Code-Credit_Product-Age-mean,Region_Code-Credit_Product-Age-median,Region_Code-Credit_Product-Age-std,Region_Code-Credit_Product-Vintage-mean,Region_Code-Credit_Product-Vintage-median,Region_Code-Credit_Product-Vintage-std,Region_Code-Credit_Product-Avg_Account_Balance-mean,Region_Code-Credit_Product-Avg_Account_Bal

In [27]:
test.sample(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Gender-Region_Code,Gender-Occupation,Gender-Channel_Code,Gender-Credit_Product,Gender-Is_Active,Region_Code-Occupation,Region_Code-Channel_Code,Region_Code-Credit_Product,Region_Code-Is_Active,Occupation-Channel_Code,Occupation-Credit_Product,Occupation-Is_Active,Channel_Code-Credit_Product,Channel_Code-Is_Active,Credit_Product-Is_Active,Gender-Age-mean,Gender-Age-median,Gender-Age-std,Gender-Vintage-mean,Gender-Vintage-median,Gender-Vintage-std,Gender-Avg_Account_Balance-mean,Gender-Avg_Account_Balance-median,Gender-Avg_Account_Balance-std,Region_Code-Age-mean,Region_Code-Age-median,Region_Code-Age-std,Region_Code-Vintage-mean,Region_Code-Vintage-median,Region_Code-Vintage-std,Region_Code-Avg_Account_Balance-mean,Region_Code-Avg_Account_Balance-median,Region_Code-Avg_Account_Balance-std,Occupation-Age-mean,Occupation-Age-median,Occupation-Age-std,Occupation-Vintage-mean,Occupation-Vintage-median,Occupation-Vintage-std,Occupation-Avg_Account_Balance-mean,Occupation-Avg_Account_Balance-median,Occupation-Avg_Account_Balance-std,Channel_Code-Age-mean,Channel_Code-Age-median,Channel_Code-Age-std,Channel_Code-Vintage-mean,Channel_Code-Vintage-median,Channel_Code-Vintage-std,Channel_Code-Avg_Account_Balance-mean,Channel_Code-Avg_Account_Balance-median,Channel_Code-Avg_Account_Balance-std,Credit_Product-Age-mean,Credit_Product-Age-median,Credit_Product-Age-std,Credit_Product-Vintage-mean,Credit_Product-Vintage-median,Credit_Product-Vintage-std,Credit_Product-Avg_Account_Balance-mean,Credit_Product-Avg_Account_Balance-median,Credit_Product-Avg_Account_Balance-std,Is_Active-Age-mean,Is_Active-Age-median,Is_Active-Age-std,Is_Active-Vintage-mean,Is_Active-Vintage-median,Is_Active-Vintage-std,Is_Active-Avg_Account_Balance-mean,Is_Active-Avg_Account_Balance-median,Is_Active-Avg_Account_Balance-std,Gender-Region_Code-Age-mean,Gender-Region_Code-Age-median,Gender-Region_Code-Age-std,Gender-Region_Code-Vintage-mean,Gender-Region_Code-Vintage-median,Gender-Region_Code-Vintage-std,Gender-Region_Code-Avg_Account_Balance-mean,Gender-Region_Code-Avg_Account_Balance-median,Gender-Region_Code-Avg_Account_Balance-std,Gender-Occupation-Age-mean,Gender-Occupation-Age-median,Gender-Occupation-Age-std,Gender-Occupation-Vintage-mean,Gender-Occupation-Vintage-median,Gender-Occupation-Vintage-std,Gender-Occupation-Avg_Account_Balance-mean,Gender-Occupation-Avg_Account_Balance-median,Gender-Occupation-Avg_Account_Balance-std,Gender-Channel_Code-Age-mean,Gender-Channel_Code-Age-median,Gender-Channel_Code-Age-std,...,Gender-Credit_Product-Avg_Account_Balance-std,Gender-Is_Active-Age-mean,Gender-Is_Active-Age-median,Gender-Is_Active-Age-std,Gender-Is_Active-Vintage-mean,Gender-Is_Active-Vintage-median,Gender-Is_Active-Vintage-std,Gender-Is_Active-Avg_Account_Balance-mean,Gender-Is_Active-Avg_Account_Balance-median,Gender-Is_Active-Avg_Account_Balance-std,Region_Code-Occupation-Age-mean,Region_Code-Occupation-Age-median,Region_Code-Occupation-Age-std,Region_Code-Occupation-Vintage-mean,Region_Code-Occupation-Vintage-median,Region_Code-Occupation-Vintage-std,Region_Code-Occupation-Avg_Account_Balance-mean,Region_Code-Occupation-Avg_Account_Balance-median,Region_Code-Occupation-Avg_Account_Balance-std,Region_Code-Channel_Code-Age-mean,Region_Code-Channel_Code-Age-median,Region_Code-Channel_Code-Age-std,Region_Code-Channel_Code-Vintage-mean,Region_Code-Channel_Code-Vintage-median,Region_Code-Channel_Code-Vintage-std,Region_Code-Channel_Code-Avg_Account_Balance-mean,Region_Code-Channel_Code-Avg_Account_Balance-median,Region_Code-Channel_Code-Avg_Account_Balance-std,Region_Code-Credit_Product-Age-mean,Region_Code-Credit_Product-Age-median,Region_Code-Credit_Product-Age-std,Region_Code-Credit_Product-Vintage-mean,Region_Code-Credit_Product-Vintage-median,Region_Code-Credit_Product-Vintage-std,Region_Code-Credit_Product-Avg_Account_Balance-mean,Region_Code-Credit_Pro

In [28]:
valid.sample(3)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,Gender-Region_Code,Gender-Occupation,Gender-Channel_Code,Gender-Credit_Product,Gender-Is_Active,Region_Code-Occupation,Region_Code-Channel_Code,Region_Code-Credit_Product,Region_Code-Is_Active,Occupation-Channel_Code,Occupation-Credit_Product,Occupation-Is_Active,Channel_Code-Credit_Product,Channel_Code-Is_Active,Credit_Product-Is_Active,Gender-Age-mean,Gender-Age-median,Gender-Age-std,Gender-Vintage-mean,Gender-Vintage-median,Gender-Vintage-std,Gender-Avg_Account_Balance-mean,Gender-Avg_Account_Balance-median,Gender-Avg_Account_Balance-std,Region_Code-Age-mean,Region_Code-Age-median,Region_Code-Age-std,Region_Code-Vintage-mean,Region_Code-Vintage-median,Region_Code-Vintage-std,Region_Code-Avg_Account_Balance-mean,Region_Code-Avg_Account_Balance-median,Region_Code-Avg_Account_Balance-std,Occupation-Age-mean,Occupation-Age-median,Occupation-Age-std,Occupation-Vintage-mean,Occupation-Vintage-median,Occupation-Vintage-std,Occupation-Avg_Account_Balance-mean,Occupation-Avg_Account_Balance-median,Occupation-Avg_Account_Balance-std,Channel_Code-Age-mean,Channel_Code-Age-median,Channel_Code-Age-std,Channel_Code-Vintage-mean,Channel_Code-Vintage-median,Channel_Code-Vintage-std,Channel_Code-Avg_Account_Balance-mean,Channel_Code-Avg_Account_Balance-median,Channel_Code-Avg_Account_Balance-std,Credit_Product-Age-mean,Credit_Product-Age-median,Credit_Product-Age-std,Credit_Product-Vintage-mean,Credit_Product-Vintage-median,Credit_Product-Vintage-std,Credit_Product-Avg_Account_Balance-mean,Credit_Product-Avg_Account_Balance-median,Credit_Product-Avg_Account_Balance-std,Is_Active-Age-mean,Is_Active-Age-median,Is_Active-Age-std,Is_Active-Vintage-mean,Is_Active-Vintage-median,Is_Active-Vintage-std,Is_Active-Avg_Account_Balance-mean,Is_Active-Avg_Account_Balance-median,Is_Active-Avg_Account_Balance-std,Gender-Region_Code-Age-mean,Gender-Region_Code-Age-median,Gender-Region_Code-Age-std,Gender-Region_Code-Vintage-mean,Gender-Region_Code-Vintage-median,Gender-Region_Code-Vintage-std,Gender-Region_Code-Avg_Account_Balance-mean,Gender-Region_Code-Avg_Account_Balance-median,Gender-Region_Code-Avg_Account_Balance-std,Gender-Occupation-Age-mean,Gender-Occupation-Age-median,Gender-Occupation-Age-std,Gender-Occupation-Vintage-mean,Gender-Occupation-Vintage-median,Gender-Occupation-Vintage-std,Gender-Occupation-Avg_Account_Balance-mean,Gender-Occupation-Avg_Account_Balance-median,Gender-Occupation-Avg_Account_Balance-std,Gender-Channel_Code-Age-mean,Gender-Channel_Code-Age-median,...,Gender-Credit_Product-Avg_Account_Balance-std,Gender-Is_Active-Age-mean,Gender-Is_Active-Age-median,Gender-Is_Active-Age-std,Gender-Is_Active-Vintage-mean,Gender-Is_Active-Vintage-median,Gender-Is_Active-Vintage-std,Gender-Is_Active-Avg_Account_Balance-mean,Gender-Is_Active-Avg_Account_Balance-median,Gender-Is_Active-Avg_Account_Balance-std,Region_Code-Occupation-Age-mean,Region_Code-Occupation-Age-median,Region_Code-Occupation-Age-std,Region_Code-Occupation-Vintage-mean,Region_Code-Occupation-Vintage-median,Region_Code-Occupation-Vintage-std,Region_Code-Occupation-Avg_Account_Balance-mean,Region_Code-Occupation-Avg_Account_Balance-median,Region_Code-Occupation-Avg_Account_Balance-std,Region_Code-Channel_Code-Age-mean,Region_Code-Channel_Code-Age-median,Region_Code-Channel_Code-Age-std,Region_Code-Channel_Code-Vintage-mean,Region_Code-Channel_Code-Vintage-median,Region_Code-Channel_Code-Vintage-std,Region_Code-Channel_Code-Avg_Account_Balance-mean,Region_Code-Channel_Code-Avg_Account_Balance-median,Region_Code-Channel_Code-Avg_Account_Balance-std,Region_Code-Credit_Product-Age-mean,Region_Code-Credit_Product-Age-median,Region_Code-Credit_Product-Age-std,Region_Code-Credit_Product-Vintage-mean,Region_Code-Credit_Product-Vintage-median,Region_Code-Credit_Product-Vintage-std,Region_Code-Credit_Product-Avg_Account_Balance-mean,Region_Code-Credit_Product-Avg_Account_Bal

In [29]:
x_train = train.drop(['ID','Is_Lead'],axis=1)
y_train = train['Is_Lead']

x_valid = valid.drop(['ID','Is_Lead'],axis=1)
y_valid = valid['Is_Lead']

x_test = test.drop(['ID'],axis=1)

In [30]:
lgb_params= {'learning_rate': 0.05, 
             'n_estimators': 500, 
             'max_bin': 100,
             'num_leaves': 10, 
             'max_depth': 30, 
             'reg_alpha': 8.5, 
             'reg_lambda': 7.0, 
             'subsample': 0.75,
            'random_state': 42,
            'class_weight':'balanced'}


lgb = LGBMClassifier(**lgb_params)
lgb.fit(x_train, y_train,eval_set=[(x_valid, y_valid)],early_stopping_rounds=10,verbose=400)
pred = lgb.predict_proba(x_valid)[:, 1]
roc_score = roc_auc_score(y_valid, pred)
print(f"roc_auc_score: {roc_score}")

Training until validation scores don't improve for 10 rounds
[400]	valid_0's binary_logloss: 0.548287
Did not meet early stopping. Best iteration is:
[500]	valid_0's binary_logloss: 0.547725
roc_auc_score: 0.7853294122805484


In [31]:
x_train = x_train.fillna(0)
x_valid = x_valid.fillna(0)
x_test = x_test.fillna(0)

In [32]:
hist_params = {'max_iter':500,
               'learning_rate' : 0.06,
               'max_depth' : 7,
               'early_stopping' : 'auto',
               'verbose':1,
               'random_state':63
              }


hist = HistGradientBoostingClassifier(**hist_params)
hist.fit(x_train,y_train)
pred = hist.predict_proba(x_valid)[:, 1]
roc_score = roc_auc_score(y_valid, pred)
print(f"roc_auc_score: {roc_score}")

Binning 0.301 GB of training data: 5.153 s
Binning 0.033 GB of validation data: 0.053 s
Fitting gradient boosted rounds:
[1/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.53582, val loss: 0.53565, in 0.531s
[2/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.52578, val loss: 0.52546, in 0.275s
[3/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.51709, val loss: 0.51670, in 0.294s
[4/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.50939, val loss: 0.50890, in 0.302s
[5/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.50272, val loss: 0.50220, in 0.327s
[6/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.49678, val loss: 0.49626, in 0.300s
[7/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.49148, val loss: 0.49098, in 0.315s
[8/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.48663, val loss: 0.48614, in 0.322s
[9/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.48235, val loss: 0.48187, in 0.313s
[10/500] 1 tree, 31 leaves, max depth = 7, train lo

[88/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.43299, val loss: 0.43654, in 0.161s
[89/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.43292, val loss: 0.43654, in 0.154s
[90/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.43283, val loss: 0.43654, in 0.183s
[91/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.43276, val loss: 0.43652, in 0.164s
[92/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.43266, val loss: 0.43650, in 0.261s
[93/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.43259, val loss: 0.43648, in 0.203s
[94/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.43251, val loss: 0.43647, in 0.184s
[95/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.43242, val loss: 0.43644, in 0.379s
[96/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.43235, val loss: 0.43642, in 0.136s
[97/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.43226, val loss: 0.43639, in 0.202s
[98/500] 1 tree, 31 leaves, max depth = 7, train loss: 0.43218, val lo

In [33]:
train_pred = pd.DataFrame()
train_pred['lgb'] = lgb.predict_proba(x_train)[:, 1]
train_pred['hist'] = hist.predict_proba(x_train)[:, 1]
train_pred.head(3)

,lgb,hist
0,0.346292,0.145404
1,0.308001,0.128163
2,0.795196,0.548271


In [34]:
valid_pred = pd.DataFrame()
valid_pred['lgb'] = lgb.predict_proba(x_valid)[:, 1]
valid_pred['hist'] = hist.predict_proba(x_valid)[:, 1]
valid_pred.head(3)

,lgb,hist
0,0.382979,0.190234
1,0.776676,0.532996
2,0.710023,0.348629


In [35]:
test_pred = pd.DataFrame()
test_pred['lgb'] = lgb.predict_proba(x_test)[:, 1]
test_pred['hist'] = hist.predict_proba(x_test)[:, 1]
test_pred.head(3)

,lgb,hist
0,0.119812,0.037360
1,0.524932,0.228525
2,0.214959,0.069202


In [36]:
model = LogisticRegression(random_state=10,max_iter=1000,class_weight='balanced',fit_intercept=True)
model.fit(train_pred,y_train)
pred = model.predict_proba(valid_pred)[:, 1]
roc_score = roc_auc_score(y_valid, pred)
print(f"roc_auc_score: {roc_score}")

roc_auc_score: 0.7855294232886535


In [37]:
pred = valid_pred.mean(axis=1)
roc_score = roc_auc_score(y_valid, pred)
print(f"roc_auc_score: {roc_score}")

roc_auc_score: 0.785601957154491


In [38]:
test_pred['stacked_avg'] = test_pred.mean(axis=1)
test_pred['stacked_logistic'] = model.predict_proba(test_pred.drop(['stacked_avg'],axis=1))[:, 1]
test_pred['ID'] = test['ID']
test_pred.head(1)

,lgb,hist,stacked_avg,stacked_logistic,ID
0,0.119812,0.03736,0.078586,0.1555,VBENBARO


In [39]:
# test_pred[['ID','stacked_avg']].rename({'stacked_avg':'Is_Lead'},axis=1).to_csv('submit_4.csv',index=False)
